In [26]:
import csv
import os
import matplotlib.pyplot as plt

# Create a directory to save the plots
output_dir = 'plots'
os.makedirs(output_dir, exist_ok=True)

# Read the data from the CSV file
data = []
with open('/home/heliya/Desktop/TextAttack-Search-Benchmark/res_csv/bert_yelp/Textattack (BERT YELP), Wordnet.csv', mode='r') as file:
    reader = csv.reader(file)
    for row in reader:
        data.append(row)

# Extract relevant data for plotting
algorithms = [row[0] for row in data[2:]]  # Skip headers and descriptions
accuracy_under_attack = [float(row[5].strip('%')) for row in data[2:]]
attack_success_rate = [float(row[6].strip('%')) for row in data[2:]]
average_perturbed_word = [float(row[7].strip('%')) for row in data[2:]]
average_num_queries = [float(row[9]) for row in data[2:]]

# Function to save plot
def save_plot(fig, filename):
    filepath = os.path.join(output_dir, filename)
    fig.savefig(filepath, bbox_inches='tight')
    plt.close(fig)

# Plotting Accuracy Under Attack
fig, ax = plt.subplots(figsize=(10, 6))
ax.barh(algorithms, accuracy_under_attack, color='skyblue')
ax.set_xlabel('Accuracy Under Attack (%)')
ax.set_ylabel('Algorithm')
ax.set_title('Accuracy Under Attack by Algorithm')
ax.grid(True, linestyle='--', alpha=0.7)
save_plot(fig, 'accuracy_under_attack.png')

# Plotting Attack Success Rate
fig, ax = plt.subplots(figsize=(10, 6))
ax.barh(algorithms, attack_success_rate, color='salmon')
ax.set_xlabel('Attack Success Rate (%)')
ax.set_ylabel('Algorithm')
ax.set_title('Attack Success Rate by Algorithm')
ax.grid(True, linestyle='--', alpha=0.7)
save_plot(fig, 'attack_success_rate.png')

# Plotting Average Perturbed Word %
fig, ax = plt.subplots(figsize=(10, 6))
ax.barh(algorithms, average_perturbed_word, color='lightgreen')
ax.set_xlabel('Average Perturbed Word (%)')
ax.set_ylabel('Algorithm')
ax.set_title('Average Perturbed Word Percentage by Algorithm')
ax.grid(True, linestyle='--', alpha=0.7)
save_plot(fig, 'average_perturbed_word.png')

# Plotting Average Number of Queries
fig, ax = plt.subplots(figsize=(10, 6))
ax.barh(algorithms, average_num_queries, color='lightcoral')
ax.set_xlabel('Average Number of Queries')
ax.set_ylabel('Algorithm')
ax.set_title('Average Number of Queries by Algorithm')
ax.grid(True, linestyle='--', alpha=0.7)
save_plot(fig, 'average_num_queries.png')


In [14]:
import pandas as pd

# Read the data from the CSV file
df_word = pd.read_csv('/home/heliya/Desktop/TextAttack-Search-Benchmark/res_csv/bert_mr/Textattack (BERT MR), Wordnet.csv')
df_embedding = pd.read_csv('/home/heliya/Desktop/TextAttack-Search-Benchmark/res_csv/bert_mr/Textattack (BERT MR), Embedding.csv')
df_hownet = pd.read_csv('/home/heliya/Desktop/TextAttack-Search-Benchmark/res_csv/bert_mr/Textattack (BERT MR), Hownet.csv')

In [15]:

# Select the relevant columns
df_word_selected = df_word[['Algorithm', 'Average Perturbed Word %', 'Average Number of Words per Input']]
df_embedding_selected = df_embedding[['Algorithm', 'Average Perturbed Word %', 'Average Number of Words per Input']]
df_hownet_selected = df_hownet[['Algorithm', 'Average Perturbed Word %', 'Average Number of Words per Input']]

# Rename the columns to identify them after the join
df_word_selected = df_word_selected.rename(columns={'Average Perturbed Word %': 'Wordnet Average Perturbed Word %', 'Average Number of Words per Input': 'Wordnet Avg Number of Words per Input'})
df_embedding_selected = df_embedding_selected.rename(columns={'Average Perturbed Word %': 'Embedding Average Perturbed Word %', 'Average Number of Words per Input': 'Embedding Avg Number of Words per Input'})
df_hownet_selected = df_hownet_selected.rename(columns={'Average Perturbed Word %': 'Hownet Average Perturbed Word %', 'Average Number of Words per Input': 'Hownet Avg Number of Words per Input'})

# Merge the dataframes
df_merged = df_word_selected.merge(df_embedding_selected, on='Algorithm').merge(df_hownet_selected, on='Algorithm')

df_merged


,Algorithm,Wordnet Average Perturbed Word %,Wordnet Avg Number of Words per Input,Embedding Average Perturbed Word %,Embedding Avg Number of Words per Input,Hownet Average Perturbed Word %,Hownet Avg Number of Words per Input
0,kmeans8,15.0%,18.38,15.0%,18.38,15.0%,18.38
1,semantic similarity4,6.59%,18.38,6.59%,18.38,6.59%,18.38
2,semantic similarity8,6.56%,18.38,6.56%,18.38,6.56%,18.38
3,kmeans,10.0%,18.38,10.0%,18.38,10.0%,18.38
4,dbscan,10.0%,18.38,10.0%,18.38,10.0%,18.38
5,tabu agglomerative ward,nan%,18.38,nan%,18.38,nan%,18.38
6,tabu agglomerative single,nan%,18.38,nan%,18.38,nan%,18.38
7,tabu agglomerative complete,nan%,18.60,nan%,18.60,nan%,18.60
8,tabu agglomerative average,nan%,18.60,nan%,18.60,nan%,18.60
9,tabu2,9.85%,18.38,9.85%,18.38,9.85%,18.38
